In [1]:
%cd ~/protein-transfer

/home/francesca/protein-transfer


In [2]:
%load_ext blackcellmagic

In [3]:
import os
import numpy as np
import pandas as pd

In [4]:
from scr.utils import pickle_load

In [5]:
opdict = pickle_load("results/pytorch-esm-onehot/structure/secondary_structure/tape_ss3_processed/onehot/noflatten/onehot-noflatten-layer_0.pkl")

In [15]:
opdict.keys()

dict_keys(['losses', 'train', 'val', 'cb513', 'ts115', 'casp12'])

In [16]:
opdict["cb513"].keys()

dict_keys(['cross-entropy', 'pred', 'true', 'acc', 'rocauc'])

In [6]:
opdict["cb513"]["acc"], opdict["cb513"]["rocauc"], opdict["ts115"]["acc"], opdict["ts115"]["rocauc"], opdict["casp12"]["acc"], opdict["casp12"]["rocauc"]

(0.48816846771958994,
 0.6405190154371231,
 0.5085510368973876,
 0.6419719552524553,
 0.48194597574421166,
 0.6202125329637339)

In [7]:
rand0 = pickle_load("results/pytorch-esm-rand/structure/secondary_structure/tape_ss3_processed/esm1_t6_43M_UR50S/noflatten/esm1_t6_43M_UR50S-noflatten-layer_0.pkl")
rand0["cb513"]["acc"], rand0["cb513"]["rocauc"], rand0["ts115"]["acc"], rand0["ts115"]["rocauc"], rand0["casp12"]["acc"], rand0["casp12"]["rocauc"]

(0.49502632308118594,
 0.6477117780057853,
 0.5146781578238621,
 0.6528848195625717,
 0.49434950385887544,
 0.6290679854668791)

In [8]:
rand1 = pickle_load("results/pytorch-esm-rand/structure/secondary_structure/tape_ss3_processed/esm1_t6_43M_UR50S/noflatten/esm1_t6_43M_UR50S-noflatten-layer_1.pkl")
rand1["cb513"]["acc"], rand1["cb513"]["rocauc"], rand1["ts115"]["acc"], rand1["ts115"]["rocauc"], rand1["casp12"]["acc"], rand1["casp12"]["rocauc"]

(0.5192297035189803,
 0.6745426123567886,
 0.5540331268516024,
 0.7032613972497685,
 0.5378996692392503,
 0.6811779345459282)

In [10]:
rand2 = pickle_load("results/pytorch-esm-rand/structure/secondary_structure/tape_ss3_processed/esm1_t6_43M_UR50S/noflatten/esm1_t6_43M_UR50S-noflatten-layer_2.pkl")
rand2["cb513"]["acc"], rand2["cb513"]["rocauc"], rand2["ts115"]["acc"], rand2["ts115"]["rocauc"], rand2["casp12"]["acc"], rand2["casp12"]["rocauc"]

(0.5189318370739817,
 0.6780248436823525,
 0.5633921357392944,
 0.7127126233866258,
 0.5465821389195149,
 0.694011705648267)

In [11]:
rand3 = pickle_load("results/pytorch-esm-rand/structure/secondary_structure/tape_ss3_processed/esm1_t6_43M_UR50S/noflatten/esm1_t6_43M_UR50S-noflatten-layer_3.pkl")
rand3["cb513"]["acc"], rand3["cb513"]["rocauc"], rand3["ts115"]["acc"], rand3["ts115"]["rocauc"], rand3["casp12"]["acc"], rand3["casp12"]["rocauc"]

(0.5187170961485176,
 0.6792022712112759,
 0.5641327767304066,
 0.7156176631336795,
 0.5501653803748622,
 0.7016838467972928)

In [12]:
rand4 = pickle_load("results/pytorch-esm-rand/structure/secondary_structure/tape_ss3_processed/esm1_t6_43M_UR50S/noflatten/esm1_t6_43M_UR50S-noflatten-layer_4.pkl")
rand4["cb513"]["acc"], rand4["cb513"]["rocauc"], rand4["ts115"]["acc"], rand4["ts115"]["rocauc"], rand4["casp12"]["acc"], rand4["casp12"]["rocauc"]

(0.5203311166528124,
 0.6804099187911713,
 0.5645704282251549,
 0.7166195765296242,
 0.5497519294377067,
 0.7039696738373027)

In [13]:
rand5 = pickle_load("results/pytorch-esm-rand/structure/secondary_structure/tape_ss3_processed/esm1_t6_43M_UR50S/noflatten/esm1_t6_43M_UR50S-noflatten-layer_5.pkl")
rand5["cb513"]["acc"], rand5["cb513"]["rocauc"], rand5["ts115"]["acc"], rand5["ts115"]["rocauc"], rand5["casp12"]["acc"], rand5["casp12"]["rocauc"]

(0.5190773067331671,
 0.6798017826181083,
 0.567330999192028,
 0.7183198091144601,
 0.5490628445424476,
 0.7025275591732285)

In [14]:
rand6 = pickle_load("results/pytorch-esm-rand/structure/secondary_structure/tape_ss3_processed/esm1_t6_43M_UR50S/noflatten/esm1_t6_43M_UR50S-noflatten-layer_6.pkl")
rand6["cb513"]["acc"], rand6["cb513"]["rocauc"], rand6["ts115"]["acc"], rand6["ts115"]["rocauc"], rand6["casp12"]["acc"], rand6["casp12"]["rocauc"]

(0.5184123025768911,
 0.6789741680936543,
 0.5685429571774845,
 0.7193889840460939,
 0.5525082690187431,
 0.7061988648684436)